In [1]:
from pbg.translate import get_port_mapping
from pbg.parse import get_process
from process_bigraph import ProcessTypes
from bigraph_schema.type_system import required_schema_keys, optional_schema_keys
from ecoli.processes.antibiotics.cell_wall import CellWall
from vivarium.core.registry import process_registry
from ecoli.processes.registries import topology_registry
from pbg.data_model.base_process import CORE
from vivarium_interface.vivarium import Vivarium

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


In [2]:
core = ProcessTypes()
types: dict = core.types()
processes = core.process_registry.registry
processes.update(process_registry.registry)
type_keys: set = required_schema_keys.union(optional_schema_keys)

In [3]:
v = Vivarium(types=types, processes=processes)

In [4]:
v.get_processes()

,Process
0,global_clock
1,ecoli-metabolism
2,replication_data_listener
3,post-division-mass-listener
4,exchange_data
5,ecoli-tf-unbinding
6,console-emitter
7,ecoli-cell-wall
8,ram-emitter
9,ecoli-metabolism-redux-classic


In [5]:
v.add_process(process_id='ecoli-cell-wall', name='cell-wall')

In [6]:
v

Vivarium( 
{ 'cell-wall': { 'address': 'local:ecoli-cell-wall',
                 'config': { 'PBP': { 'PBP1A': 'CPLX0-7717[m]',
                                      'PBP1B_alpha': 'CPLX0-3951[i]',
                                      'PBP1B_gamma': 'CPLX0-8300[c]'},
                             'cell_radius': 0.5,
                             'critical_radius': 20,
                             'disaccharide_height': 1.03,
                             'disaccharide_width': 1.4,
                             'inter_strand_distance': 0.6,
                             'max_expansion': 3,
                             'murein': 'CPD-12261[p]',
                             'seed': 0,
                             'strand_term_p': 0.07675577224377576,
                             'time_step': 10,
                             'timestep': 10},
                 'inputs': {},
                 'instance': <ecoli.processes.antibiotics.cell_wall.CellWall object at 0x13bd7a870>,
                 'inte

In [15]:
y = x.tolist()

In [16]:
type(y)

float